In [1]:
import torch
import gradio as gr
import numpy as np
from torchvision import transforms
from sklearn.preprocessing import StandardScaler
import torch.nn as nn
from torchvision.models import resnet18
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
import dicomsdl
import tensorflow as tf
from PIL import Image
import cv2
from sklearn.preprocessing import StandardScaler


# Configuración del modelo preentrenado VGG16 y tamaño de imagen
RESIZE_TO = (512, 512)

# Modelo preentrenado VGG16 para heatmaps
vgg16_model = VGG16(weights="imagenet")
grad_model = Model(inputs=vgg16_model.inputs, outputs=[vgg16_model.get_layer("block5_conv3").output, vgg16_model.output])

# Cargar el modelo local
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define tu clase de modelo combinado (ajusta si ya tienes esta definición)
class CombinedModel(nn.Module):
    def __init__(self, num_features, num_classes=2):
        super(CombinedModel, self).__init__()

        # Rama de imágenes: ResNet18 preentrenado
        self.resnet = resnet18(pretrained=True)
        num_resnet_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Identity()  # Eliminar la capa final para usar embeddings

        # Rama de características tabulares
        self.tabular_branch = nn.Sequential(
            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3)
        )

        # Capa combinada
        self.combined_branch = nn.Sequential(
            nn.Linear(num_resnet_features + 64, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, image, features):
        # Procesar imágenes
        image_features = self.resnet(image)
        # Procesar características tabulares
        tabular_features = self.tabular_branch(features)
        # Combinar ambas ramas
        combined_features = torch.cat((image_features, tabular_features), dim=1)
        # Predicción final
        output = self.combined_branch(combined_features)

        return output

# Cargar modelo local
model = CombinedModel(num_features=4)  # Número correcto de características
model.load_state_dict(torch.load("cancer_classification_model_final.pth", map_location=device, weights_only=True))
model.to(device)


c:\Users\Melissa\Ambientes\venv_integrador_2\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Melissa\Ambientes\venv_integrador_2\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Melissa\Ambientes\venv_integrador_2\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


CombinedModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, trac

In [2]:
import pandas as pd
train_df = pd.read_csv('train.csv')

minority_class_size = train_df['cancer'].value_counts().min()

# Definir la proporción de muestras de la clase mayoritaria (ejemplo: 2 veces la clase minoritaria)
majority_class_multiplier = 10
majority_class_size = minority_class_size * majority_class_multiplier

# Obtener todas las muestras de la clase minoritaria
minority_samples = train_df[train_df['cancer'] == 1]

# Obtener una muestra aleatoria de la clase mayoritaria
majority_samples = train_df[train_df['cancer'] == 0].sample(majority_class_size, random_state=42)

# Combinar las muestras de ambas clases
balanced_df = pd.concat([minority_samples, majority_samples]).sample(frac=1, random_state=42).reset_index(drop=True)

base_path='images_processed_heatmap'

# saving image path into train dataframe
balanced_df['img_path']= balanced_df.patient_id.astype(str)\
                    + '/' + balanced_df.image_id.astype(str)\
                    + '.png'


base_path='images_processed_heatmap'

# saving image path into train dataframe
balanced_df['img_path']= balanced_df.patient_id.astype(str)\
                    + '/' + balanced_df.image_id.astype(str)\
                    + '.png'

from sklearn.model_selection import train_test_split
balanced_df['label'] = balanced_df['cancer']  # Usar 'cancer' como la etiqueta

# Dividir en conjuntos de entrenamiento y validación
train_df_mlo, val_df_mlo = train_test_split(balanced_df, test_size=0.2, random_state=42,stratify=balanced_df['label'] )

training_tabular_features = train_df_mlo[['age', 'biopsy', 'invasive', 'BIRADS']]


# Durante el entrenamiento
mean = training_tabular_features.mean(axis=0)  # Media de cada columna
std = training_tabular_features.std(axis=0)    # Desviación estándar de cada columna

print("Medias:", mean)
print("Desviaciones estándar:", std)

Medias: age         58.897939
biopsy       0.121492
invasive     0.063003
BIRADS       0.710136
dtype: float64
Desviaciones estándar: age         10.061221
biopsy       0.326714
invasive     0.242980
BIRADS       0.606114
dtype: float64


In [3]:
# Transformaciones de imágenes
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Configurar el normalizador
scaler = StandardScaler()
scaler.fit(training_tabular_features)

# Medias y desviaciones estándar calculadas durante el entrenamiento
mean = np.array([58, 0.1, 0.0, 0.7])  # Ejemplo
std = np.array([10, 0.3, 0.2, 0.6])  # Ejemplo

# Función para preprocesar las variables tabulares
def preprocess_user_input(age, biopsy, invasive, BIRADS):
    user_input = np.array([age, biopsy, invasive, BIRADS])
    # Normalizar usando las estadísticas del entrenamiento
    normalized_input = (user_input - mean) / std
    # Convertir a tensor
    return torch.tensor(normalized_input, dtype=torch.float32).to(device)

def process_image(image, age, density, invasive, BIRADS):
    try:
        # Determinar el formato de la imagen cargada
        if hasattr(image, "filename") and image.filename.lower().endswith(".dcm"):
            temp_dcm_path = "temp_image.dcm"
            image.save(temp_dcm_path)
            processed_ary = dicom_file_to_ary(temp_dcm_path)
            image = Image.fromarray(processed_ary)

        # Generar heatmap y superponerlo
        heatmap = generate_heatmap(image)
        transformed_image = apply_heatmap(image, heatmap)

        # Convertir la imagen transformada a un tensor
        transformed_image_tensor = transform(Image.fromarray(cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)))
        transformed_image_tensor = transformed_image_tensor.unsqueeze(0).to(device)

        # Preprocesar las variables tabulares del usuario
        tabular_data = preprocess_user_input(age, density, invasive, BIRADS).unsqueeze(0) 

        model.eval()
        # Realizar la predicción
        with torch.no_grad():
            outputs = model(transformed_image_tensor, tabular_data)
            probabilities = torch.softmax(outputs, dim=1)
            class_idx = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0, class_idx].item()

        # Clases de salida
        classes = ["No se evidencia Cáncer", "Se evidencia malignidad, remitir a estudios adicionales"]
        prediction = classes[class_idx]

        # Convertir la imagen transformada con el heatmap para visualización
        transformed_image_pil = Image.fromarray(cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB))

        # return transformed_image_pil, f"Predicción: {prediction} (Confianza: {confidence:.2f})"
        return transformed_image_pil, f"Predicción: {prediction} (Confianza: {confidence:.2f})"
    except Exception as e:
        return "Error", str(e)

# Función para convertir DICOM a PNG
def dicom_file_to_ary(path):
    dcm_file = dicomsdl.open(str(path))
    data = dcm_file.pixelData()

    data = np.array(data, dtype=np.float32)
    data = (data - data.min()) / (data.max() - data.min())
    data = (data * 255).astype(np.uint8)

    # Redimensionar al tamaño especificado
    image = Image.fromarray(data)
    image = image.resize(RESIZE_TO, Image.LANCZOS)
    return np.array(image, dtype=np.uint8)

# Función para generar heatmap
def generate_heatmap(image):
    image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    image_resized = cv2.resize(image_cv, (224, 224))
    img_array = np.expand_dims(image_resized, axis=0)
    img_array = tf.keras.applications.vgg16.preprocess_input(img_array)

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = tf.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)[0]
    weights = tf.reduce_mean(grads, axis=(0, 1))
    heatmap = tf.reduce_sum(weights * conv_outputs[0], axis=-1)

    # Normalizar heatmap
    heatmap = tf.maximum(heatmap, 0) / tf.reduce_max(heatmap)
    if isinstance(heatmap, tf.Tensor):  # Convertir a NumPy si es necesario
        heatmap = heatmap.numpy()
    heatmap = cv2.resize(heatmap, (image_cv.shape[1], image_cv.shape[0]))
    heatmap = (heatmap * 255).astype("uint8")
    return heatmap

# Función para superponer heatmap
def apply_heatmap(image, heatmap, alpha=0.6):
    image_cv = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(heatmap_color, alpha, image_cv, 1 - alpha, 0)
    return overlay




In [ ]:
# Interfaz de Gradio
interface = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(type="pil"),  # Entrada como imagen PIL
        gr.Number(label="Edad"),  # Variable: Edad
        gr.Number(label="Biopsia (1=Sí, 0=No)"),  # Variable: Biopsia
        gr.Number(label="Invasivo (1=Sí, 0=No)"),  # Variable: Invasivo
        gr.Number(label="BIRADS (0, 1 o 2 )")  # Variable: BIRADS
    ],
    outputs=[
        gr.Image(type="pil", label="Imagen Transformada"),  # Imagen con heatmap
        gr.Text(label="Predicción"),  # Predicción del modelo
    ],
    title="Detección de Cáncer con Imágenes Radiológicas",
    description=(
        "Esta herramienta fue creada con el propósito de contribuir a la detección temprana del cáncer de seno," 
        " apoyando a profesionales médicos en su misión de salvar vidas. A través del análisis de imágenes radiológicas y variables clínicas," 
        " este sistema combina la precisión de las redes neuronales con la pasión por el cuidado humano." 
        " Subiendo una imagen y proporcionando información adicional, podrás obtener una predicción que puede marcar la diferencia en el diagnóstico y tratamiento de los pacientes." 
        " Nuestro objetivo es ser un aliado en la lucha contra el cáncer, porque cada esfuerzo cuenta y cada vida importa."
    )
)

interface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://1c4a791e97c4ca96a3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


c:\Users\Melissa\Ambientes\venv_integrador_2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(
c:\Users\Melissa\Ambientes\venv_integrador_2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(
c:\Users\Melissa\Ambientes\venv_integrador_2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(
c:\Users\Melissa\Ambientes\venv_integrador_2\Lib\site-packages\keras\src\models\functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['keras_tensor']. Received: the structure of inputs=*
  warnings.warn(
